In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MaxAbsScaler


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

import numbers
from sklearn import model_selection

In [2]:
# Aquí, a futuro, habría que añadir los datos del Nasdaq, oro y/o Ethereum para ver los resultados
data_for_use = pd.read_csv('./bitcoin_data_for_use_v4.csv', index_col=0)
data_gold = pd.read_csv('./gold_data_for_use.csv', index_col=0)
data_nasdaq = pd.read_csv('./nasdaq_data_for_use.csv', index_col=0)

#dia y outlier ya hemos visto que no sirven de mucho (o de nada)
data_for_use_w_vol= data_for_use.drop(labels=['dia', 'outlier'], axis=1)

data_for_use_w_gold = data_for_use_w_vol.merge(data_gold, left_index=True, right_index=True)
data_for_use_w_nasdaq = data_for_use_w_vol.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_w_all = data_for_use_w_gold.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_basic = data_for_use.drop(labels=['dia',
                    'varV0','varV1','varV2','varV3','varV4','varV5','varV6','varV7','varV8','varV9',
                    'varV10','varV11','varV12','varV13','varV14','varV15','varV16','varV17','varV18','varV19',
                    'varV20','varV21','varV22','varV23','varV24','varV25','varV26','varV27','varV28','varV29', 'outlier'
                    ], axis=1)

data_for_use_basic_w_gold = data_for_use_basic.merge(data_gold, left_index=True, right_index=True)
data_for_use_basic_w_nasdaq = data_for_use_basic.merge(data_nasdaq, left_index=True, right_index=True)
data_for_use_basic_w_all = data_for_use_basic_w_gold.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_bone_deep = data_for_use_basic.drop(labels=['varPm93',
                    'varPm123','varPm153','varPm183','varPm213','varPm243','varPm273','varPm303','varPm333',
                    'varPs30','varPs37','varPs44','varPs51','varPs58','varPs65','varPs72','varPs79','varPs86'
                    ], axis=1)

data = []
data.append(['data_for_use_w_vol', data_for_use_w_vol])
data.append(['data_for_use_w_gold', data_for_use_w_gold])
data.append(['data_for_use_w_nasdaq', data_for_use_w_nasdaq])
data.append(['data_for_use_w_all', data_for_use_w_all])
data.append(['data_for_use_basic', data_for_use_basic])
data.append(['data_for_use_basic_w_gold', data_for_use_basic_w_gold])
data.append(['data_for_use_basic_w_nasdaq', data_for_use_basic_w_nasdaq])
data.append(['data_for_use_basic_w_all', data_for_use_basic_w_all])
data.append(['data_for_use_bone_deep', data_for_use_bone_deep])

In [3]:
modelos=[]
modelos.append(['LogisticRegression_none_newton-cg', LogisticRegression(penalty='none', solver='newton-cg', random_state=0)])
modelos.append(['LogisticRegression_none_lbfgs', LogisticRegression(penalty='none', solver='lbfgs', random_state=0)])
modelos.append(['LogisticRegression_none_sag', LogisticRegression(penalty='none', solver='sag', random_state=0)])
modelos.append(['LogisticRegression_none_saga', LogisticRegression(penalty='none', solver='saga', random_state=0)])
modelos.append(['LogisticRegression_l2_newton-cg', LogisticRegression(penalty='l2', solver='newton-cg', random_state=0)])
modelos.append(['LogisticRegression_l2_lbfgs', LogisticRegression(penalty='l2', solver='lbfgs', random_state=0)])
modelos.append(['LogisticRegression_l2_liblinear', LogisticRegression(penalty='l2', solver='liblinear', random_state=0)])
modelos.append(['LogisticRegression_l2_sag', LogisticRegression(penalty='l2', solver='sag', random_state=0)])
modelos.append(['LogisticRegression_l2_saga', LogisticRegression(penalty='l2', solver='saga', random_state=0)])
modelos.append(['LogisticRegression_l1_liblinear', LogisticRegression(penalty='l1', solver='liblinear', random_state=0)])
modelos.append(['LogisticRegression_l1_saga', LogisticRegression(penalty='l1', solver='saga', random_state=0)])
modelos.append(['LogisticRegression_elasticnet_saga', LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga', random_state=0)])
for vecinos in range(1,31):
    modelos.append(['KNeighbors_'+str(vecinos), KNeighborsClassifier(n_neighbors=vecinos)])
for grado in range(1,4):
    modelos.append(['SVC_'+str(grado), SVC(probability=True, kernel='poly', degree=grado, random_state=0)])
for profund in range(1,31):
    modelos.append(['DecisionTree_'+str(profund), DecisionTreeClassifier(max_depth=profund, random_state=0)])
for estimadores in range(1,21):
    modelos.append(['AdaBoost_'+str(estimadores), AdaBoostClassifier(n_estimators=estimadores, random_state=0)])
for min_muestras in range(2,21):
    modelos.append(['RandomForest_'+str(min_muestras), RandomForestClassifier(min_samples_split=min_muestras, random_state=0)])

In [4]:
def walk_forward_validation (model, X_train_wfv, y_train_wfv, X_test_wfv, y_test_wfv, real_test_wfv, ct_wfv):
    y_pred_wfv = list()
    #variable que indica si, siguiendo el modelo, estaríamos invertidos o no
    dentro=True
    resultado_naive=1
    resultado_mod=1
    resultado_mod_comis=1
    
    
    for i in range(len(y_test_wfv)):
        X_train_wfv_ct=ct_wfv.fit_transform(X_train_wfv)
        model.fit(X_train_wfv_ct, y_train_wfv)
        X_test_wfv_ct=ct_wfv.transform(X_test_wfv)
        y_pred_next = model.predict(X_test_wfv_ct[i:i+1])
        y_pred_wfv.append(y_pred_next[0])
        X_train_wfv=X_train_wfv.append(X_test_wfv[i:i+1])
        y_train_wfv=y_train_wfv.append(pd.Series(y_test_wfv[i]))
        
        
        #suponemos una comisión por compra/venta del 0,15%
        if y_pred_next[0]!=dentro:
            resultado_mod_comis=resultado_mod_comis*(1-0.0015)
        dentro=y_pred_next[0]
        
        #el naive seimpre se mantiene comprado (o siempre supone que sube)
        resultado_naive=resultado_naive*(1+real_test_wfv[i])
        #si estamos comprados entonces se nos aplica la subida o bajada del día siguiente
        if dentro:
            resultado_mod=resultado_mod*(1+real_test_wfv[i])
            resultado_mod_comis=resultado_mod_comis*(1+real_test_wfv[i])
    
    return metrics.confusion_matrix(y_test_wfv, y_pred_wfv), metrics.f1_score(y_test_wfv,y_pred_wfv), \
        metrics.accuracy_score(y_test_wfv,y_pred_wfv), resultado_naive, resultado_mod, resultado_mod_comis

In [5]:
resultados=[]

#bucle de datos
for d in data:
    data_wfv = d[1]
    data_train, data_test = train_test_split(data_wfv, shuffle=False, test_size=0.01)
    
    # Separamos la variable que queremos predecir
    X_train=data_train.drop(labels=['subida', 'varPSig'], axis=1)
    y_train=data_train['subida']
    real_train=data_train['varPSig']

    X_test=data_test.drop(labels=['subida', 'varPSig'], axis=1)
    y_test=data_test['subida']
    real_test=data_test['varPSig']
    
    #litado de scalers

    number_columns = X_train.select_dtypes('number').columns

    tSS=[
        #('dia', 
        #OneHotEncoder(handle_unknown='ignore'),
        #['dia']),
        ('scaler', StandardScaler(),number_columns)
        ]
    tMM=[
        ('scaler', MinMaxScaler(),number_columns)
        ]
    #tPTbc=[
    #    ('scaler', PowerTransformer(method='box-cox', standardize=False),number_columns)
    #    ]
    tPTyj=[
        ('scaler', PowerTransformer(method='yeo-johnson', standardize=False),number_columns)
        ]
    tQT=[
        ('scaler', QuantileTransformer(output_distribution='normal', random_state=0),number_columns)
        ]
    tMA=[
        ('scaler', MaxAbsScaler(),number_columns)
        ]

    ct_list=[]
    ct_list.append(['StandardScaler', ColumnTransformer(transformers=tSS, remainder='passthrough')])
    ct_list.append(['MinMaxScaler', ColumnTransformer(transformers=tMM, remainder='passthrough')])
    #ct_list.append(['PowerTransformer_box-cox', ColumnTransformer(transformers=tPTbc, remainder='passthrough')])
    ct_list.append(['PowerTransformer_yeo-johnson', ColumnTransformer(transformers=tPTyj, remainder='passthrough')])
    ct_list.append(['QuantileTransformer', ColumnTransformer(transformers=tQT, remainder='passthrough')])
    ct_list.append(['MaxAbsScaler', ColumnTransformer(transformers=tMA, remainder='passthrough')])
    
    #bucle de scalers
    for c in ct_list:
        #bucle de modelos
        for m in modelos:
            print(d[0] + ' ' + c[0] + ' ' + m[0])
            conf_mat, f1, acc, res_naive, res_mod, res_comis = \
                walk_forward_validation (m[1], X_train, y_train, X_test, y_test, real_test, c[1])
            resultados.append([d[0], c[0], m[0], conf_mat, f1, acc, res_naive, res_mod, res_comis])

data_for_use_w_vol StandardScaler LogisticRegression_none_newton-cg
data_for_use_w_vol StandardScaler LogisticRegression_none_lbfgs
data_for_use_w_vol StandardScaler LogisticRegression_none_sag
data_for_use_w_vol StandardScaler LogisticRegression_none_saga
data_for_use_w_vol StandardScaler LogisticRegression_l2_newton-cg
data_for_use_w_vol StandardScaler LogisticRegression_l2_lbfgs
data_for_use_w_vol StandardScaler LogisticRegression_l2_liblinear
data_for_use_w_vol StandardScaler LogisticRegression_l2_sag
data_for_use_w_vol StandardScaler LogisticRegression_l2_saga
data_for_use_w_vol StandardScaler LogisticRegression_l1_liblinear
data_for_use_w_vol StandardScaler LogisticRegression_l1_saga
data_for_use_w_vol StandardScaler LogisticRegression_elasticnet_saga
data_for_use_w_vol StandardScaler KNeighbors_1
data_for_use_w_vol StandardScaler KNeighbors_2
data_for_use_w_vol StandardScaler KNeighbors_3
data_for_use_w_vol StandardScaler KNeighbors_4
data_for_use_w_vol StandardScaler KNeighbors

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

data_for_use_w_vol MinMaxScaler LogisticRegression_none_sag
data_for_use_w_vol MinMaxScaler LogisticRegression_none_saga


/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_it

...

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_it

data_for_use_bone_deep MinMaxScaler LogisticRegression_l2_newton-cg
data_for_use_bone_deep MinMaxScaler LogisticRegression_l2_lbfgs
data_for_use_bone_deep MinMaxScaler LogisticRegression_l2_liblinear
data_for_use_bone_deep MinMaxScaler LogisticRegression_l2_sag
data_for_use_bone_deep MinMaxScaler LogisticRegression_l2_saga
data_for_use_bone_deep MinMaxScaler LogisticRegression_l1_liblinear
data_for_use_bone_deep MinMaxScaler LogisticRegression_l1_saga
data_for_use_bone_deep MinMaxScaler LogisticRegression_elasticnet_saga
data_for_use_bone_deep MinMaxScaler KNeighbors_1
data_for_use_bone_deep MinMaxScaler KNeighbors_2
data_for_use_bone_deep MinMaxScaler KNeighbors_3
data_for_use_bone_deep MinMaxScaler KNeighbors_4
data_for_use_bone_deep MinMaxScaler KNeighbors_5
data_for_use_bone_deep MinMaxScaler KNeighbors_6
data_for_use_bone_deep MinMaxScaler KNeighbors_7
data_for_use_bone_deep MinMaxScaler KNeighbors_8
data_for_use_bone_deep MinMaxScaler KNeighbors_9
data_for_use_bone_deep MinMaxSca

data_for_use_bone_deep PowerTransformer_yeo-johnson KNeighbors_26
data_for_use_bone_deep PowerTransformer_yeo-johnson KNeighbors_27
data_for_use_bone_deep PowerTransformer_yeo-johnson KNeighbors_28
data_for_use_bone_deep PowerTransformer_yeo-johnson KNeighbors_29
data_for_use_bone_deep PowerTransformer_yeo-johnson KNeighbors_30
data_for_use_bone_deep PowerTransformer_yeo-johnson SVC_1
data_for_use_bone_deep PowerTransformer_yeo-johnson SVC_2
data_for_use_bone_deep PowerTransformer_yeo-johnson SVC_3
data_for_use_bone_deep PowerTransformer_yeo-johnson DecisionTree_1
data_for_use_bone_deep PowerTransformer_yeo-johnson DecisionTree_2
data_for_use_bone_deep PowerTransformer_yeo-johnson DecisionTree_3
data_for_use_bone_deep PowerTransformer_yeo-johnson DecisionTree_4
data_for_use_bone_deep PowerTransformer_yeo-johnson DecisionTree_5
data_for_use_bone_deep PowerTransformer_yeo-johnson DecisionTree_6
data_for_use_bone_deep PowerTransformer_yeo-johnson DecisionTree_7
data_for_use_bone_deep Powe

data_for_use_bone_deep QuantileTransformer DecisionTree_8
data_for_use_bone_deep QuantileTransformer DecisionTree_9
data_for_use_bone_deep QuantileTransformer DecisionTree_10
data_for_use_bone_deep QuantileTransformer DecisionTree_11
data_for_use_bone_deep QuantileTransformer DecisionTree_12
data_for_use_bone_deep QuantileTransformer DecisionTree_13
data_for_use_bone_deep QuantileTransformer DecisionTree_14
data_for_use_bone_deep QuantileTransformer DecisionTree_15
data_for_use_bone_deep QuantileTransformer DecisionTree_16
data_for_use_bone_deep QuantileTransformer DecisionTree_17
data_for_use_bone_deep QuantileTransformer DecisionTree_18
data_for_use_bone_deep QuantileTransformer DecisionTree_19
data_for_use_bone_deep QuantileTransformer DecisionTree_20
data_for_use_bone_deep QuantileTransformer DecisionTree_21
data_for_use_bone_deep QuantileTransformer DecisionTree_22
data_for_use_bone_deep QuantileTransformer DecisionTree_23
data_for_use_bone_deep QuantileTransformer DecisionTree_24

data_for_use_bone_deep MaxAbsScaler AdaBoost_16
data_for_use_bone_deep MaxAbsScaler AdaBoost_17
data_for_use_bone_deep MaxAbsScaler AdaBoost_18
data_for_use_bone_deep MaxAbsScaler AdaBoost_19
data_for_use_bone_deep MaxAbsScaler AdaBoost_20
data_for_use_bone_deep MaxAbsScaler RandomForest_2
data_for_use_bone_deep MaxAbsScaler RandomForest_3
data_for_use_bone_deep MaxAbsScaler RandomForest_4
data_for_use_bone_deep MaxAbsScaler RandomForest_5
data_for_use_bone_deep MaxAbsScaler RandomForest_6
data_for_use_bone_deep MaxAbsScaler RandomForest_7
data_for_use_bone_deep MaxAbsScaler RandomForest_8
data_for_use_bone_deep MaxAbsScaler RandomForest_9
data_for_use_bone_deep MaxAbsScaler RandomForest_10
data_for_use_bone_deep MaxAbsScaler RandomForest_11
data_for_use_bone_deep MaxAbsScaler RandomForest_12
data_for_use_bone_deep MaxAbsScaler RandomForest_13
data_for_use_bone_deep MaxAbsScaler RandomForest_14
data_for_use_bone_deep MaxAbsScaler RandomForest_15
data_for_use_bone_deep MaxAbsScaler Rand

In [6]:
resultados

[['data_for_use_w_vol',
  'StandardScaler',
  'LogisticRegression_none_newton-cg',
  array([[2, 9],
         [7, 3]]),
  0.2727272727272727,
  0.23809523809523808,
  0.8348925980482585,
  0.7664382166487861,
  0.7607071490359],
 ['data_for_use_w_vol',
  'StandardScaler',
  'LogisticRegression_none_lbfgs',
  array([[2, 9],
         [7, 3]]),
  0.2727272727272727,
  0.23809523809523808,
  0.8348925980482585,
  0.7664382166487861,
  0.7607071490359],
 ['data_for_use_w_vol',
  'StandardScaler',
  'LogisticRegression_none_sag',
  array([[2, 9],
         [7, 3]]),
  0.2727272727272727,
  0.23809523809523808,
  0.8348925980482585,
  0.7664382166487861,
  0.7607071490359],
 ['data_for_use_w_vol',
  'StandardScaler',
  'LogisticRegression_none_saga',
  array([[2, 9],
         [7, 3]]),
  0.2727272727272727,
  0.23809523809523808,
  0.8348925980482585,
  0.7664382166487861,
  0.7607071490359],
 ['data_for_use_w_vol',
  'StandardScaler',
  'LogisticRegression_l2_newton-cg',
  array([[2, 9],
     

In [8]:
resulados_df = pd.DataFrame(resultados)

In [9]:
resulados_df.to_csv('prueba_bucle_resultados.csv')